In [81]:
import json
import pandas as pd 

In [82]:
def load_data(name):
    # Opening JSON file
    f = open(name)
    data = json.load(f)

    return data

def save_to_csv (repo,path_save,name_save,i_max,i_min):
    name_i = lambda i: f'navigation_data ({i}).txt' 
    dict_df = []
    for i in range(i_min,i_max + 1):
        data_json = load_data(repo + name_i(i))
        # get the name of the last saved action
        last_action = list(data_json.keys())[-1]
        # get the data of the last saved action
        dict_df += [list(data_json[last_action].values())]

    df = pd.DataFrame(dict_df,columns=list(data_json[last_action].keys()))
    df.to_csv(path_save + name_save)

In [83]:
# CAN BE MODIFIED----------------------------------

repo = 'C:\\Users\\mtlsa\\Documents\\' 
path_save = 'C:\\Users\\mtlsa\\Documents\\'
name_save = 'csv_data.csv'

# rename navigation_data into navigation_data (0)
i_max = 6
i_min = 4

#----------------------------------------------------


In [84]:
save_to_csv(repo,path_save,name_save,i_max,i_min)